In [2]:
'''
    涉及到的知识: KNN, 数据归一化方法，划分数据集和训练集
'''
import pandas as pd

In [4]:
oriData = pd.read_table('datingTestSet.txt', header=None)

In [11]:
oriData.max()

0         91273
1       20.9193
2       1.69552
3    smallDoses
dtype: object

In [16]:
oriData.index[11]

11

### 数据归一化

0-1标准化， Z-score标准化, Sigmoid压缩法

In [8]:
'''
0-1标准化
data = (X - min)/(max - min)
'''
def minmax(dataSet):
    minDf = dataSet.min()
    maxDf = dataSet.max()
    normSet = (dataSet - minDf)/(maxDf - minDf)
    
    return normSet


### 划分训练集和测试集

In [18]:
'''
功能：切分训练集和测试集
参数说明：
    dataSet: 原始数据集
    rate: 训练集所占比例
返回：
    划分好的训练集和测试集
'''

# 要保证数据集是乱序的
def randSplit(dataSet, rate):
    num = dataSet.shape[0]
    spnum = int(num*rate)
    train = dataSet.iloc[:spnum, :]
    test = dataSet.iloc[spnum:, :]
    test.index = range(test.shape[0])
    
    return train, test
    
    # test的索引不是从0开始的，所以要修改

In [40]:
'''
函数功能：KNN分类，输出正确率
参数说明：
    train:训练集
    test: 测试集
    k: knn参数
返回: 预测好分类的测试集
'''

def datingClass(train, test, k):
    n = train.shape[1] - 1
    m = test.shape[0]
    result = []
    
    for i in range(m):
        dist = list((((train.iloc[:, :n] - test.iloc[i, :n])**2).sum(1))**0.5)
        dist_1 = pd.DataFrame({'dist': dist, 'labels': (train.iloc[:, n])})
        dr = dist_1.sort_values(by='dist')[:k]
        re = dr.loc[:, 'labels'].value_counts()
        result.append(re.index[0])
    
    result = pd.Series(result)
    test.loc[:, 'predict'] = result
    acc = (test.iloc[:, -1] == test.iloc[:, -2]).mean()
    
    print("模型准确率为:{}".format(acc))
    
    return test

In [41]:
dating = pd.concat([minmax(oriData.iloc[:, :3]), oriData.iloc[:, 3]], axis=1)

In [42]:
dating.head()

,0,1,2,3
0,0.448325,0.398051,0.562334,largeDoses
1,0.158733,0.341955,0.987244,smallDoses
2,0.285429,0.068925,0.474496,didntLike
3,0.823201,0.628480,0.252489,didntLike
4,0.420102,0.079820,0.078578,didntLike


In [43]:
train, test = randSplit(dating, 0.92)

In [44]:
res = datingClass(train, test, 5)

模型准确率为:0.9625


d:\python\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
d:\python\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [46]:
res.head()

,0,1,2,3,predict
0,0.748370,0.508872,0.408589,didntLike,didntLike
1,0.342511,0.461926,0.897321,largeDoses,largeDoses
2,0.380770,0.515810,0.774052,largeDoses,largeDoses
3,0.146900,0.134351,0.129138,smallDoses,smallDoses
4,0.332683,0.469709,0.818801,largeDoses,largeDoses


### hamming 距离
等长字符串，相同位置不同元素个数
Levenshtein包